In [5]:
word_list = nltk.corpus.words.words()

In [20]:
len(data)

8689

In [18]:
five_letter_words = []
for word in word_list:
    if len(word) == 5:
        if word[0] == word[0].lower():
            five_letter_words.append(word)


In [19]:
import pandas as pd

word_series = []
char_series = []
for word in five_letter_words:
    word = word.lower()
    word_series.append(word)
    char_series.append([word[0], word[1], word[2], word[3], word[4]])
data = pd.DataFrame(data=char_series, index=word_series)

In [8]:
import seaborn as sns

In [10]:
def notIn(location, letter, dataset):
    filtered_dataset = dataset[dataset[location]!=letter]
    return filtered_dataset

def In(location, letter, dataset):
    filtered_dataset = dataset[dataset[location]==letter]
    return filtered_dataset

def InSomewhereElse(location, letter, dataset):
    filtered_dataset = notIn(location, letter, dataset)
    for i in range(5):
        filtered_dataset = pd.concat([filtered_dataset, In(i, letter, filtered_dataset)]).drop_duplicates()
    return filtered_dataset

In [31]:
import nltk
import pandas as pd
import statistics

def scoreFirstGuess(guess, word_set=None):
    if word_set is None:
        word_list = nltk.corpus.words.words()
        five_letter_words = []
        word_series = []
        char_series = []
        for word in word_list:
            if len(word) == 5:
                if word[0] == word[0].lower():
                    word_series.append(word)
                    char_series.append([word[0], word[1], word[2], word[3], word[4]])
        word_set = pd.DataFrame(char_series, index=word_series)
    
    prior = 1 / len(word_set.index)

    p_series = []
    counter = 1
    for answer in word_set.index:
        filtered_data = word_set
        for i in range(5):
            m = ''
            if guess[i] == answer[i]:
                m = 'G'
                filtered_data = In(i, guess[i], filtered_data)
            elif guess[i] in answer and guess[i] != answer[i]:
                m = 'Y'
                filtered_data = InSomewhereElse(i, guess[i], filtered_data)
            else:
                m = 'B'
                filtered_data = notIn(i, guess[i], filtered_data)
        prob = 1 / len(filtered_data.index)
        delta_prob = prob - prior
        p_series.append(delta_prob)
        average = statistics.mean(p_series)
        mean = statistics.median(p_series)
    return [guess, p_series, average, mean]

def solveFirstGuess(data = None):
    if data is None:
        word_list = nltk.corpus.words.words()

        # Get 5 Letter Words
        five_letter_words = []
        word_series = []
        char_series = []
        for word in word_list:
            word = word.lower()
            if len(word) == 5:
                word_series.append(word)
                char_series.append([word[0], word[1], word[2], word[3], word[4]])

        data = pd.DataFrame(char_series, index=word_series)

    prior = 1 / len(data.index)

    p_series = []
    average_series = []
    mean_series = []
    counter = 1
    for guess in data.index:
        print("{} / {} : {}".format(counter, len(data.index), counter/len(data.index)*100))
        p = []
        for answer in data.index:
            filtered_data = data
            for i in range(5):
                m = ''
                if guess[i] == answer[i]:
                    m = 'G'     
                    filtered_data = In(i, guess[i], filtered_data)
                elif guess[i] in answer and guess[i] != answer[i]:
                    m = 'Y'
                    filtered_data = InSomewhereElse(i, guess[i], filtered_data)
                else:
                    m = 'B'
                    filtered_data = notIn(i, guess[i], filtered_data)
            prob = 1 / len(filtered_data.index)
            delta_prob = prob - prior
            p.append(delta_prob)
        counter += 1
        p_series.append(p)
        average_series.append(statistics.mean(p))
        mean_series.append(statistics.median(p))
    data['Score'] = p_series
    data['Average Score'] = average_series
    data['Median Score'] = mean_series
    return data


In [37]:
test_single  = scoreFirstGuess(data.index[0], data)

In [14]:
import multiprocessing as mp
test_data = data[0:10]


In [43]:
data.index

Index(['aalii', 'abaca', 'aback', 'abaff', 'abaft', 'abase', 'abash', 'abask',
       'abate', 'abave',
       ...
       'watch', 'water', 'wheel', 'where', 'while', 'white', 'woman', 'wound',
       'wrong', 'young'],
      dtype='object', length=8689)

In [42]:
pool_size = mp.cpu_count() - 2
test_data = data[0:pool_size]
pool = mp.Pool(mp.cpu_count() - 2) # Set CPU 
try: 
    result = pool.starmap(scoreFirstGuess, [(word, data) for word in test_data]) 
except:
    print('error')
pool.close()

Process SpawnPoolWorker-56:
Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'scoreFirstGuess' on <module '__main__' (built-in)>
Process SpawnPoolWorker-54:
Traceback (most recent call last):
  File "/Application

KeyboardInterrupt: 

1. 6 guesses
2. 5 Spaces with 3 states: 
    1. Not Known
    2. Known
    3. Partially Known 
3. Each guess gives:
    Upto 5 new information.
    1. letters not used
    2. letters used in the correct place
    3. letters used but not in the correct place

Objective: "solve" Wordle

Algorithm: 
Score = delta p
p = answer / possible guesses
